In [ ]:
import sys 
sys.path.append('/neurospin/meg/meg_tmp/Calculation_Pedro_2014/scripts/decoding')
from calc_dec_wTask_CR import calc_dec_wTask_CR
gat, score, diagonal = calc_dec_wTask_CR('/neurospin/meg/meg_tmp/Calculation_Pedro_2014',['opbigdec', 'opbigdec'],'s01','class')

/neurospin/meg/meg_tmp/Calculation_Pedro_2014/scripts/decoding/calc_dec_wTask_CR.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  info_calc_delay['operand'] = info_calc_delay['preResult'] # add another column 'operand' for the big decoder
/neurospin/meg/meg_tmp/Calculation_Pedro_2014/scripts/decoding/calc_dec_wTask_CR.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  info_calc_nodelay['operand'] = info_calc_nodelay['preResult'] # add another column 'operand' for the big decoder


362 matching events found
0 bad epochs dropped
concat well
<EpochsArray  |  n_events : 362 (all good), tmin : -0.5 (s), tmax : 4.5 (s), baseline : None>
     run  operand1  operator  operand2  preResult  delay  corrResult  deviant  \
0      1         5         1         2          6      1           7       -1   
1      1         3        -1         1          3      0           2        1   
2      1         5         0        33          1      0           5       -4   
3      1         5         1         2          6      1           7       -1   
4      1         6        -1         2          7      1           4        3   
5      1         4        -1         2          1      0           2       -1   
6      1         6         1         3          5      0           9       -4   
7      1         4         0        33          4      1           4        0   
8      1         5         0        33          1      0           5       -4   
9      1         6        -1         

/volatile/anaconda/lib/python2.7/site-packages/mne/epochs.py:394: UserWarning: The measurement information indicates a low-pass frequency of 125 Hz. The decim=100 parameter will result in a sampling frequency of 2.5 Hz, which can cause aliasing artifacts.
  % (lowpass, decim, new_sfreq))  # > 50% nyquist limit


In [ ]:
%debug

In [ ]:
import sys 
sys.path.append('/neurospin/meg/meg_tmp/Calculation_Pedro_2014/scripts/decoding')
from fldtrp2mne_calc import fldtrp2mne_calc
import numpy as np
import mne
import pandas as pd

In [ ]:
data_path = '/neurospin/meg/meg_tmp/Calculation_Pedro_2014/data/mat'
Subject = 's01'

In [ ]:
fname_calc = data_path + '/' + Subject + '_calc_smooth.mat'
epoch_calc,info_calc = fldtrp2mne_calc(fname_calc, 'data', 'calc')


In [ ]:
idx_delay = info_calc['delay'] == 1
idx_nodelay = info_calc['delay'] == 0
epoch_calc_delay = epoch_calc[idx_delay]
epoch_calc_nodelay = epoch_calc[idx_nodelay]
epoch_calc_Op1 = epoch_calc[info_calc['operand1'] < 100] # have to add that otherwise it rewrites info_calc!-why?
epoch_calc_Op2 = epoch_calc[info_calc['operand1'] < 100] 

info_calc_delay = info_calc[info_calc['delay'] == 1]
info_calc_nodelay = info_calc[info_calc['delay'] == 0]
info_calc_Op1 = info_calc[info_calc['operand1'] < 100]
info_calc_Op2 = info_calc[info_calc['operand1'] < 100]

time_calc_crop = np.arange(-0.1,0.8004,0.004) 

epoch_calc_delay.crop(3.5,4.4)
epoch_calc_delay.times = time_calc_crop

epoch_calc_nodelay.crop(3.1,4)
epoch_calc_nodelay.times = time_calc_crop

info_calc_delay['operand'] = info_calc_delay['preResult'] # add another column 'operand' for the big decoder
info_calc_nodelay['operand'] = info_calc_nodelay['preResult'] # add another column 'operand' for the big decoder

epoch_calc_resplock = mne.epochs.concatenate_epochs([epoch_calc_delay,epoch_calc_nodelay])
info_calc_resplock = pd.concat([info_calc_delay, info_calc_nodelay])
print 'concat well'
# Create the big decoder with operand1, operand2 and presented result 
# Epochs will have 900 ms, -100 to 800. 
# Epochs Operand1
epoch_calc_Op1.crop(-0.1,0.8)
epoch_calc_Op1.times = time_calc_crop
info_calc_Op1['operand'] = info_calc_Op1['operand1'] # add another column 'operand' for the big decoder
print epoch_calc
print info_calc
# Epochs Operand1
epoch_calc_Op2.crop(0.7, 1.6)
epoch_calc_Op2.times = time_calc_crop
info_calc_Op2['operand'] = info_calc_Op2['operand2'] # add another column 'operand' for the big decoder
# Concatenate
epoch_calc_bigdec = mne.epochs.concatenate_epochs([epoch_calc_Op1,epoch_calc_Op2,epoch_calc_delay,epoch_calc_nodelay])	
info_calc_bigdec = pd.concat([info_calc_Op1,info_calc_Op2,info_calc_delay, info_calc_nodelay])


In [ ]:
train_index = info_calc_bigdec['operator'] != 0
X_train = epoch_calc_bigdec[train_index]
y_train = np.array(info_calc_bigdec[train_index]['operand'])
y_train = y_train.astype(np.float64)
X_test = X_train
y_test = y_train

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
print np.asarray((unique, counts)).T

In [ ]:
idx_delay = info_calc['delay'] == 1
idx_nodelay = info_calc['delay'] == 0

info_calc_delay = info_calc[info_calc['delay'] == 1]
info_calc_nodelay = info_calc[info_calc['delay'] == 0]

time_calc_crop = np.arange(-0.1,0.8004,0.004) 

epoch_calc_delay = epoch_calc[idx_delay]
epoch_calc_delay.crop(3.5,4.4)
epoch_calc_delay.times = time_calc_crop

epoch_calc_nodelay = epoch_calc[idx_nodelay]
epoch_calc_nodelay.crop(3.1,4)
epoch_calc_nodelay.times = time_calc_crop

info_calc_delay['operand'] = info_calc_delay['preResult'] # add another column 'operand' for the big decoder
info_calc_nodelay['operand'] = info_calc_nodelay['preResult'] # add another column 'operand' for the big decoder

epoch_calc_resplock = mne.epochs.concatenate_epochs([epoch_calc_delay,epoch_calc_nodelay])
info_calc_resplock = pd.concat([info_calc_delay, info_calc_nodelay])



In [ ]:
idx_delay = info_calc['delay'] == 1
idx_nodelay = info_calc['delay'] == 0

info_calc_delay = info_calc[info_calc['delay'] == 1]
info_calc_nodelay = info_calc[info_calc['delay'] == 0]

time_calc_crop = np.arange(-0.1,0.8004,0.004) 

epoch_calc_delay = epoch_calc[idx_delay]
epoch_calc_delay.crop(3.5,4.4)
epoch_calc_delay.times = time_calc_crop

epoch_calc_nodelay = epoch_calc[idx_nodelay]
epoch_calc_nodelay.crop(3.1,4)
epoch_calc_nodelay.times = time_calc_crop

info_calc_delay['operand'] = info_calc_delay['preResult'] # add another column 'operand' for the big decoder
info_calc_nodelay['operand'] = info_calc_nodelay['preResult'] # add another column 'operand' for the big decoder

epoch_calc_resplock = mne.epochs.concatenate_epochs([epoch_calc_delay,epoch_calc_nodelay])
info_calc_resplock = pd.concat([info_calc_delay, info_calc_nodelay])


In [ ]:
# Epochs Operand1
epoch_calc_Op1 = epoch_calc
epoch_calc_Op1.crop(-1,0.8)
info_calc_Op1['operand'] = info_calc_Op1['operand1'] # add another column 'operand' for the big decoder

epoch_calc_Op2 = epoch_calc
epoch_calc_Op2.crop(0.7, 1.6)
info_calc_Op2['operand'] = info_calc_Op2['operand2'] # add another column 'operand' for the big decoder

epoch_calc_bigdec = mne.epochs.concatenate_epochs([epoch_calc_Op1,epoch_calc_Op2,epoch_calc_delay,epoch_calc_nodelay])	
info_calc_bigdec = pd.concat([info_calc_Op1,info_calc_Op2,info_calc_delay, info_calc_nodelay])

In [ ]:
epoch_calc

In [ ]:
idx_delay = info_calc['delay'] == 1
idx_nodelay = info_calc['delay'] == 0
time_calc_crop = np.arange(-0.1,0.8004,0.004)



In [ ]:
epoch_calc_delay = epoch_calc[idx_delay]
epoch_calc_delay.crop(3.5,4.4)
epoch_calc_delay.times = time_calc_crop


In [ ]:
epoch_calc_nodelay = epoch_calc[idx_nodelay]
epoch_calc_nodelay.crop(3.1,4)
epoch_calc_nodelay.times = time_calc_crop

In [ ]:
epoch_calc_resplock = mne.epochs.concatenate_epochs([epoch_calc_delay,epoch_calc_nodelay])

In [ ]:
epoch_calc_res

In [ ]:
baseline = (-0.5, -0.05) #time for the baseline period
epoch_calc.apply_baseline(baseline)
evoked_baseline = epoch_calc.average()
evoked_baseline.plot()

In [ ]:
%pylab inline 
evoked.plot()
evoked_baseline.plot()

In [ ]:
info_calc_delay = info_calc[info_calc['delay'] == 1]
info_calc_nodelay = info_calc[info_calc['delay'] == 0]


In [ ]:
info_calc_resplock = pd.concat([info_calc_delay, info_calc_nodelay])

In [ ]:
X_train = epoch_calc_resplock[(info_calc_resplock['preResult'] >= 3) & (info_calc_resplock['preResult'] <= 6)]

In [ ]:
y_train = np.array(info_calc_resplock['preResult'])
y_train = y_train.astype(np.float64)
X_test = X_train
y_test = y_train

In [ ]:
y_train


In [ ]:
b

In [ ]:
%pylab qt
a.plot()


In [ ]:
c = mne.epochs.concatenate_epochs([a,b])

In [ ]:
c

In [ ]:
c.plot()